In [13]:
import os
import pandas as pd

# Пути к файлам
input_folder = r"C:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\chembl_uniprot_ttd\chembl\Результаты расчёта"
output_file = "summary_report.csv"

# Собираем файлы для анализа
files = [f for f in os.listdir(input_folder) if os.path.isfile(os.path.join(input_folder, f))]

# Создаем список для хранения данных
results = []

# Обрабатываем каждый файл
for file in files:
    file_path = os.path.join(input_folder, file)
    df = pd.read_csv(file_path, encoding='utf-8')
    
    # Собираем метрики
    metrics = {
        'File Name': file,
        'Total Rows': len(df),
        'Yes in Drug Mechanism': len(df[df["Drug Mechanism ChEMBL"] == "Yes"]),
        'Approved': len(df[df["Max phase ChEMBL"] == "Approved"]),
        'Phase 1': len(df[df["Max phase ChEMBL"] == "Phase 1"]),
        'Phase 2': len(df[df["Max phase ChEMBL"] == "Phase 2"]),
        'Phase 3': len(df[df["Max phase ChEMBL"] == "Phase 3"]),
        'Unique Genes': len(df["Gene"].unique()),
        'Genes with Yes in Mechanism': len(df[df["Drug Mechanism ChEMBL"] == "Yes"]["Gene"].unique()),
        'Genes': list(df[df["Drug Mechanism ChEMBL"] == "Yes"]["Gene"].unique())
    }
    results.append(metrics)

# Создаем основной DataFrame
summary_df = pd.DataFrame(results)

# Транспонируем таблицу
transposed = summary_df.set_index('File Name').T
transposed.index.name = 'Metrics'

# Подготовка данных для сохранения
output_lines = []
headers = ['Metrics'] + list(transposed.columns)
output_lines.append(','.join(headers))

# Добавляем основные метрики (все кроме списка генов)
for metric in transposed.index[:-1]:
    row = [metric] + list(transposed.loc[metric].astype(str))
    output_lines.append(','.join(row))

# Обрабатываем список генов
max_genes = max(len(x) for x in summary_df['Genes'])

# Добавляем строку "Genes list" с первыми генами
genes_row = ['Genes list']
for genes in summary_df['Genes']:
    genes_row.append(genes[0] if len(genes) > 0 else '')
output_lines.append(','.join(genes_row))

# Добавляем остальные гены
for i in range(1, max_genes):
    row = ['']
    for genes in summary_df['Genes']:
        row.append(genes[i] if i < len(genes) else '')
    output_lines.append(','.join(row))

# Сохраняем в CSV файл
with open(output_file, 'w', encoding='utf-8') as f:
    f.write('\n'.join(output_lines))

print(f"Сводный файл сохранен как {output_file}")

Сводный файл сохранен как summary_report.csv
